<a href="https://colab.research.google.com/github/map72ru/biz_learn/blob/main/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Для нашего пайплайна (Case1) поэкспериментировать с разными моделями: 1 - бустинг, 2 - логистическая регрессия (не забудьте здесь добавить в cont_transformer стандартизацию - нормирование вещественных признаков)
2. Отобрать лучшую модель по метрикам (кстати, какая по вашему мнению здесь наиболее подходящая DS-метрика)
3. Для отобранной модели (на отложенной выборке) сделать оценку экономической эффективности при тех же вводных, как в вопросе 2 (1 доллар на привлечение, 2 доллара - с каждого правильно классифицированного (True Positive) удержанного). (подсказка) нужно посчитать FP/TP/FN/TN для выбранного оптимального порога вероятности и посчитать выручку и траты.
4. (опционально) Провести подбор гиперпараметров лучшей модели по итогам 2-3
5. (опционально) Еще раз провести оценку экономической эффективности

In [ ]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/hw2')

Mounted at /content/hw2


In [ ]:
df = pd.read_csv("hw2/MyDrive/data/churn_data.csv")
print(df.shape)
df.head(3)

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1


In [ ]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df, df['Exited'], random_state=0)

In [ ]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [ ]:
categorical_columns = ['Geography', 'Gender', 'Tenure', 'HasCrCard', 'IsActiveMember']
continuous_columns = ['CreditScore', 'Age', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [ ]:
final_transformers = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col))
            ])
    final_transformers.append((cont_col, cont_transformer))

In [ ]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(random_state = 42)),
])

In [ ]:
#обучим наш пайплайн
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('Geography',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))],
                                                          verbose=False)),
                                                ('Gender',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                     

In [ ]:
#наши прогнозы для тестовой выборки
preds = pipeline.predict_proba(X_test)[:, 1]
preds[:10]

array([0.28502357, 0.22926788, 0.1982218 , 0.08930468, 0.04798288,
       0.80662596, 0.03834531, 0.05054472, 0.31038377, 0.73455865])

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.408508, F-Score=0.646, Precision=0.704, Recall=0.597


Вопрос 1: объясните своими словами смысл метрик Precison, Recall *
Какова их взаимосвязь и как с ними связан порог вероятности?
Можно ли подобрать порог так, что recall будет равен 1? Что при этом будет с precision
Аналогичный вопрос про precision

Высокое значение precision означает, что модель правильно предсказывает класс 1 по отношению к ложным срабатываниям (TN). В свою очередь recall говорит о полноте определения класса 1. Т.е. сколько было правильно предсказано экземпляров класса 1 к их общему количеству. Связаны данные метрики, на мой взгляд, через объем правильно определенных элементов класса 1. При росте правильных ответов класса 1 precision будет более быстро стремиться к 1, чем recall, который так же будет увеличиваться. Чем меньше порог тем recall выше и, соответственно, чем выше порог, тем выше точность. Т.о. при пороге равном 0 выбираются все эл-ты класса 1, в т.ч. и с ложным срабатыванием (FN). И наоборот (для precision) при пороге равном 1 precision будет равен 1, но при этом recall будет стремиться к нулю.

In [ ]:
from sklearn.preprocessing import StandardScaler
final_transformersLC = list()

for cat_col in categorical_columns:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformersLC.append((cat_col, cat_transformer))
    
for cont_col in continuous_columns:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('normalizator', StandardScaler())
            ])
    final_transformersLC.append((cont_col, cont_transformer))

In [ ]:
featsLC = FeatureUnion(final_transformersLC)

feature_processingLC = Pipeline([('feats', featsLC)])

In [ ]:
pipelineLC = Pipeline([
    ('features',featsLC),
    ('classifier', LogisticRegression(random_state = 42)),
])

In [ ]:
pipelineLC.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('Geography',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))],
                                                          verbose=False)),
                                                ('Gender',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                     

In [ ]:
predsLC = pipelineLC.predict_proba(X_test)[:, 1]
predsLC[:10]

array([0.22805865, 0.3350657 , 0.15347886, 0.12466446, 0.15507743,
       0.6431308 , 0.06214346, 0.077948  , 0.36717132, 0.76751542])

In [ ]:
precisionLC, recallLC, thresholdsLC = precision_recall_curve(y_test, predsLC)

fscoreLC = (2 * precisionLC * recallLC) / (precisionLC + recallLC)
# locate the index of the largest f score
ixLC = np.argmax(fscoreLC)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholdsLC[ixLC], 
                                                                        fscoreLC[ixLC],
                                                                        precisionLC[ixLC],
                                                                        recallLC[ixLC]))

Best Threshold=0.289522, F-Score=0.510, Precision=0.462, Recall=0.568


Лучший результат дает модель градиентного бустинга. И оценивать, на мой взгляд, лучше по precision, поскольку по данной метрике лучше считать экономику.

In [45]:
cm = confusion_matrix(y_test, pipeline.predict(X_test))

In [46]:
print(cm)

[[1901   90]
 [ 245  264]]


Т.к. задача поставлена не совсем понятно, то посчитаем экономику только на компанию по оттоку

In [47]:
1901*2 - 1901 - 245*2

1411

В случае "полной" экономики

In [56]:
1901*2 - 1901 - 245*2 + 2*264 - 2*90

1759

Экономический эффект от удержания (при условии, что все которые в оттоке не уйдут) будет 1411 доллар, т.е. экономически эффективно.

In [54]:
from sklearn.model_selection import GridSearchCV

params={'classifier__learning_rate':[0.1, 0.2, 0.3],
        'classifier__n_estimators':[100, 150, 200],
        'classifier__max_depth':[3, 2, 4],
        'classifier__min_samples_split': [2, 3, 4],
        'classifier__min_samples_leaf': [1, 2, 3]
        }

In [55]:
grid = GridSearchCV(pipeline,
                    param_grid=params,
                    cv=6,
                    refit=False)

search = grid.fit(X_train, y_train)
search.best_params_

{'classifier__learning_rate': 0.1,
 'classifier__max_depth': 3,
 'classifier__min_samples_leaf': 1,
 'classifier__min_samples_split': 2,
 'classifier__n_estimators': 200}

In [57]:
pipeline = Pipeline([
    ('features',feats),
    ('classifier', GradientBoostingClassifier(learning_rate=0.1, n_estimators=200, max_depth=3, 
                                          min_samples_split=2, min_samples_leaf=1, random_state=42)),
])
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('Geography',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FeatureSelector(column='Geography')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Geography'))],
                                                          verbose=False)),
                                                ('Gender',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  FeatureSelector(column='Gender')),
                                     

In [58]:
predsLC = pipeline.predict_proba(X_test)[:, 1]
predsLC[:10]

array([0.22127685, 0.22070006, 0.17692048, 0.0914358 , 0.0300503 ,
       0.84157537, 0.03656306, 0.05761089, 0.2872654 , 0.69875765])

In [59]:
precision, recall, thresholds = precision_recall_curve(y_test, preds)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.408508, F-Score=0.646, Precision=0.704, Recall=0.597


In [60]:
cm = confusion_matrix(y_test, pipeline.predict(X_test))

In [61]:
print(cm)

[[1897   94]
 [ 239  270]]


In [62]:
print(1897*2 - 1897 - 239*2)
print(1897*2 - 1897 - 239*2 + 2*70 - 2*94)

1419
1371
